In [1]:
"""
Collaborative Filtering ALS Recommender System using Spark MLlib adapted from
the Spark Summit 2014 Recommender System training example.

Developed By: Pranav Masariya
updated By: Prachi Hada
Supervisor: Dr. Magdalini Eirinaki
"""

import os
import numpy as np
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from pyspark.mllib.recommendation import ALS
from pyspark.ml.recommendation import ALS as mlals
from pyspark.ml.evaluation import RegressionEvaluator

import math
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.ml.evaluation import RegressionEvaluator

In [2]:
# Calling spark session to register application
spark = SparkSession \
    .builder \
    .appName("Recom") \
    .config("spark.recom.demo", "1") \
    .getOrCreate()

In [3]:
"""
Loading and Parsing Dataset
    Each line in the ratings dataset (ratings.csv) is formatted as:
         userId,movieId,rating,timestamp
    Each line in the movies (movies.csv) dataset is formatted as:
        movieId,title,genres

""" 

# Load ratings
ratings_df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("D:\\SJSU_courses\\256_Large_Scale_Analytics\\40 marks assignment\\PartC\\movies.csv")

In [4]:
ratings_df

DataFrame[userId: int, movieId: int, rating: int]

In [5]:
"""
For the simplicity of this tutorial
    For each line in the ratings dataset, we create a tuple of (UserID, MovieID, Rating). 
    We drop the timestamp because we do not need it for this recommender.
"""

#ratings_df = ratings_df.drop('timestamp')
ratings_df.show(30)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|   149|      2|     5|
|   149|      3|     3|
|   149|      5|     3|
|   149|      7|     2|
|   149|      9|     4|
|   149|     10|     4|
|   149|     11|     4|
|   149|     12|     4|
|   149|     13|     4|
|   149|     14|     3|
|   149|     16|     5|
|   149|     18|     4|
|   149|     20|     5|
|   149|     21|     4|
|   969|      5|     5|
|   969|     14|     3|
|   969|     15|     1|
|   969|     16|     1|
|   969|     21|     2|
|   589|      1|     4|
|   589|      2|     5|
|   589|      3|     5|
|   589|      4|     4|
|   589|      5|     3|
|   589|      6|     3|
|   589|      7|     2|
|   589|      8|     4|
|   589|      9|     5|
|   589|     10|     5|
|   589|     11|     3|
+------+-------+------+
only showing top 30 rows



In [6]:
# Load movies
movies_df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("D:\\SJSU_courses\\256_Large_Scale_Analytics\\40 marks assignment\\PartC\\movies21.csv")

In [7]:
movies_df.show(5)

+-------+--------------------+
|movieId|               title|
+-------+--------------------+
|      1|Rogue One / Star ...|
|      2|          Fight Club|
|      3|   Lord of the Rings|
|      4|              Trolls|
|      5|       Despicable Me|
+-------+--------------------+
only showing top 5 rows



In [8]:
"""
For each line in the movies dataset, we create a tuple of (MovieID, Title). 
    We drop the genres because we do not use them for this recommender.
"""
#Since the dataset which I created did not have genres, I did not have to drop it
#movies_df = movies_df.drop('genres')
#movies_df.show(5)

'\nFor each line in the movies dataset, we create a tuple of (MovieID, Title). \n    We drop the genres because we do not use them for this recommender.\n'

In [42]:
"""
In order to determine the best ALS parameters, we will use the small dataset. 
We need first to split it into train, validation, and test datasets.
"""
(trainingData,validationData,testData) = ratings_df.randomSplit([0.8,0.1,0.1])

In [43]:
# Prepare test and validation set. They should not have ratings

validation_for_predict = validationData.select('userId','movieId')
test_for_predict = testData.select('userId','movieId')

In [44]:
"""
Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by 
using Alternating Least Squares. The implementation in MLlib has the following parameters:

    1. numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
    2. rank is the number of latent factors in the model.
    3. iterations is the number of iterations to run.
    4. lambda specifies the regularization parameter in ALS.
    5. implicitPrefs specifies whether to use the explicit 
        feedback ALS variant or one adapted for implicit feedback data.
    6. alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline 
        confidence in preference observations.

"""

seed = 5 #Random seed for initial matrix factorization model. A value of None will use system time as the seed.
iterations = 10
regularization_parameter = 0.1 #run for different lambdas - e.g. 0.01
#ranks = [4, 8, 12] #number of features
ranks = [12, 15, 20] #number of features
errors = [0, 0, 0]
#models = [0, 0, 0] #To fnd models#
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1

In [45]:
# Let us traing our dataset and check the best rank with lowest RMSE
# predictAll method of the ALS takes only RDD format and hence we need to convert our dataframe into RDD
# df.rdd will automatically converts Dataframe into RDD

for rank in ranks:
    model = ALS.train(trainingData, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict.rdd).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validationData.rdd.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean()) # RMSE Error
    #print("err %d" %(err))
    #print("errors %d"%(errors)
    errors[err] = error
    #models[err] = model
    err += 1
    print ('For rank %s the RMSE is %s' % (rank, error))
    if error < min_error:
        min_error = error
        best_rank = rank
    

print ('The best model was trained with rank %s' % best_rank)

For rank 12 the RMSE is 1.0058873144591214
For rank 15 the RMSE is 0.995047166913738
For rank 20 the RMSE is 0.987580514978236
The best model was trained with rank 20


In [46]:
model = ALS.train(trainingData, best_rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)

In [47]:
"""
Spark will soon deprecate MLLIb package. 
They are focusing more on ML packages with standard machine learning implementation
Let's see that package also
"""
als =  mlals(maxIter=iterations,rank=4,seed=seed,regParam=regularization_parameter, userCol="userId", itemCol="movieId",ratingCol="rating")
modelML = als.fit(trainingData)
pred = modelML.transform(validationData)
pred = pred.where(pred['prediction'] != 'NaN')
    
# Evaluate the model by computing RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(pred)

print ('RMSE is %s' % rmse)

"""
The best part is we do not have to worry about RDD any more with this library
"""

RMSE is 0.9985192323226102


'\nThe best part is we do not have to worry about RDD any more with this library\n'

In [48]:
# Let's take test dataset and get ratings
predictions_test = model.predictAll(test_for_predict.rdd).map(lambda r: ((r[0], r[1]), r[2]))

In [49]:
## visualize preditions, here third element is predictions generated by ALS Model
predictions_test.take(3)

[((200, 16), 2.4236910016183333),
 ((200, 20), 4.356737311120778),
 ((200, 21), 3.758480495816861)]

In [50]:
"""
Let's start recommending movies.
I have written a method to call recommendations for a perticular user from test data

TODO: You need to execute one more step before calling getRecommendations, 
      Think about that step. If you go through the seps below, you will realize it soon. - 
"""
def getRecommendations(user,testDf,trainDf,model):
    # get all user and his/her rated movies
    userDf = testDf.filter(testDf.userId == user)
    # filter movies from main set which have not been rated by selected user
    # and pass it to model we sreated above
    mov = trainDf.select('movieId').subtract(userDf.select('movieId'))
    
    # Again we need to covert our dataframe into RDD
    pred_rat = model.predictAll(mov.rdd.map(lambda x: (user, x[0]))).collect()
    
    # Get the top 5 recommendations
    recommendations = sorted(pred_rat, key=lambda x: x[2], reverse=True)[:5]
    
    return recommendations

In [51]:
# Assign user id for which we need recommendations
user = 858

# Call getRecommendations method
derived_rec = getRecommendations(user,ratings_df,trainingData,model)

print ("Movies recommended for:%d" % user)



Movies recommended for:858


In [52]:
derived_rec

[Rating(user=858, product=2, rating=4.840835691412698),
 Rating(user=858, product=12, rating=4.666836407620735),
 Rating(user=858, product=9, rating=4.653912486440795),
 Rating(user=858, product=10, rating=4.5528879725719555),
 Rating(user=858, product=19, rating=4.427450639981219)]

In [53]:
# Print the result
# TODO: we can convert derived_rec into a dataframe to present it properly
for i in range(len(derived_rec)):
    print (i+1)
    movies_df.filter(movies_df.movieId==derived_rec[i][1]).select('title').show()

1
+----------+
|     title|
+----------+
|Fight Club|
+----------+

2
+------------+
|       title|
+------------+
|Pulp Fiction|
+------------+

3
+----------------+
|           title|
+----------------+
|The big Lebowski|
+----------------+

4
+-------------+
|        title|
+-------------+
|Almost Famous|
+-------------+

5
+------------+
|       title|
+------------+
|Ghostbusters|
+------------+



In [54]:
df_rec = spark.sparkContext.parallelize(derived_rec).toDF()

In [55]:
df_rec.show()

+----+-------+------------------+
|user|product|            rating|
+----+-------+------------------+
| 858|      2| 4.840835691412698|
| 858|     12| 4.666836407620735|
| 858|      9| 4.653912486440795|
| 858|     10|4.5528879725719555|
| 858|     19| 4.427450639981219|
+----+-------+------------------+



In [56]:
rec_movies = df_rec.join(movies_df,df_rec.product == movies_df.movieId,"inner").select('user','movieId','title','rating')

In [57]:
rec_movies.show()

+----+-------+----------------+------------------+
|user|movieId|           title|            rating|
+----+-------+----------------+------------------+
| 858|      2|      Fight Club| 4.840835691412698|
| 858|     12|    Pulp Fiction| 4.666836407620735|
| 858|      9|The big Lebowski| 4.653912486440795|
| 858|     10|   Almost Famous|4.5528879725719555|
| 858|     19|    Ghostbusters| 4.427450639981219|
+----+-------+----------------+------------------+

